# Analyzing Hyperpartisan Documents

## 1. Setup Working Environment

### 1.1. Load zip data files

#### 1.1.1. Download zip files from Google Drive link

In [1]:
import gdown

zip_data_url = "https://drive.google.com/drive/folders/1e8tgF2tGdZJ0HU-7pKX7yMLBLw71XTMe?usp=drive_link"
gdown.download_folder(zip_data_url)

Retrieving folder contents


Processing file 11XnbNY-7LdzEoo-HLaTzQcnFy4CVBPPZ articles-validation-bypublisher-20181122.zip
Processing file 193XKdPA2rq5oq1KNKZdp7-jCDcjE9rOb ground-truth-validation-bypublisher-20181122.zip


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=11XnbNY-7LdzEoo-HLaTzQcnFy4CVBPPZ
From (redirected): https://drive.google.com/uc?id=11XnbNY-7LdzEoo-HLaTzQcnFy4CVBPPZ&confirm=t&uuid=b3d2f4d0-b8bb-4d2e-ba30-12f235b42efa
To: /home/jfu08/PycharmProjects/AnalyzingHyperpartisanDocuments/notebooks/data/articles-validation-bypublisher-20181122.zip
100%|██████████| 337M/337M [01:48<00:00, 3.11MB/s] 
Downloading...
From: https://drive.google.com/uc?id=193XKdPA2rq5oq1KNKZdp7-jCDcjE9rOb
To: /home/jfu08/PycharmProjects/AnalyzingHyperpartisanDocuments/notebooks/data/ground-truth-validation-bypublisher-20181122.zip
100%|██████████| 5.24M/5.24M [00:01<00:00, 3.17MB/s]
Download completed


['/home/jfu08/PycharmProjects/AnalyzingHyperpartisanDocuments/notebooks/data/articles-validation-bypublisher-20181122.zip',
 '/home/jfu08/PycharmProjects/AnalyzingHyperpartisanDocuments/notebooks/data/ground-truth-validation-bypublisher-20181122.zip']

#### 1.1.2. Move downloaded data files to the corresponding `zip_data_path`

In [2]:
import os
import shutil


downloaded_data_path = 'data'
zip_data_path = '../data/zip'

os.makedirs(zip_data_path)
for zip_data_file in os.listdir('data'):
    source_path = os.path.join(downloaded_data_path, zip_data_file)
    destination_path = os.path.join(zip_data_path, zip_data_file)
    shutil.move(source_path, destination_path)

os.rmdir(downloaded_data_path)

### 1.3. Install required packages

In [3]:
!pip install -r ../requirements.txt

## 2. Perform Hyperpartisan Documents Analysis 

### 2.1 Add necessary import statements

In [4]:
from src.log_odd_ratios.LogOddRatiosAnalyzer import LogOddRatiosAnalyzer
from src.log_odd_ratios.LogOddRatiosCalculator import LogOddRatiosCalculator, TokenType
from src.constant_values.enums import DocumentType
from src.get_hyperpartisan_data.HyperpartisanDocumentsFormatter import HyperpartisanDocumentsFormatter
from src.get_hyperpartisan_data.HyperpartisanDocumentsProcessor import HyperpartisanDocumentsProcessor
from src.utils import console_output_formatter

### 2.2. Adapt hyperpartisan documents format into text files

In [5]:
console_output_formatter.print_section_header(section_header='Data formatting')
hyperpartisan_documents_formatter = HyperpartisanDocumentsFormatter()
hyperpartisan_documents_formatter.adapt_dataset_format()

###########################
#                         #
#     DATA FORMATTING     #
#                         #
###########################

Extracting XML data from zip files ...
Extracting file articles-validation-bypublisher-20181122.zip ...
File articles-validation-bypublisher-20181122.zip extracted
Extracting file ground-truth-validation-bypublisher-20181122.zip ...
File ground-truth-validation-bypublisher-20181122.zip extracted


Extracting text from articles ...: 100%|██████████| 150000/150000 [00:24<00:00, 6011.95it/s]

### 2.3. Process hyperpartisan documents (if necessary) and load

In [6]:
console_output_formatter.print_section_header(section_header='Data loading / preprocessing')
hyperpartisan_documents_processor = HyperpartisanDocumentsProcessor()

hyperpartisan_document_list = hyperpartisan_documents_processor.get_clean_documents(
    document_type=DocumentType.HYPERPARTISAN
)
console_output_formatter.print_document_list_stats(
    document_list=hyperpartisan_document_list,
    document_type=DocumentType.HYPERPARTISAN
)

non_hyperpartisan_document_list = hyperpartisan_documents_processor.get_clean_documents(
    document_type=DocumentType.NON_HYPERPARTISAN
)
console_output_formatter.print_document_list_stats(
    document_list=non_hyperpartisan_document_list,
    document_type=DocumentType.NON_HYPERPARTISAN
)

########################################
#                                      #
#     DATA LOADING / PREPROCESSING     #
#                                      #
########################################


Getting clean documents from txt files for HYPERPARTISAN ...: 100%|██████████| 75000/75000 [03:50<00:00, 325.56it/s]


Clean document list (HYPERPARTISAN) loaded successfully.
Saving HYPERPARTISAN document list into pickle file ...
Pickle file saved. 

DOCUMENT LIST STATISTICS (HYPERPARTISAN):
Number of documents: 75000
Document sample after performing preprocessing:
['pro-life', 'students', 'wilfrid', 'laurier', 'university', 'say', 'longer', 'able', 'put', 'pro-life', 'flag', 'display', ',', 'campus', 'leaders', 'explaining', 'exhibits', 'traumatic', 'one', 'last', 'fall', 'caused', 'uproar', 'canadian', 'campus', '.', 'display', 'used', '10,000', 'pink', 'blue', 'flags', 'represent', '100,000', 'abortions', 'year', 'canada', '.', 'student', 'union', 'president', 'tyler', 'van', 'herzele', ',', 'letter', 'shared', 'campuswide', ',', 'stated', '“', 'visceral', 'nature', 'display', 'interfered', 'ability', 'members', 'laurier', 'community', 'safely', 'attend', 'class', ',', 'travel', 'work', 'remain', 'campus', 'study.', '”', '“', 'adversarial', 'tone', 'event', 'evoked', 'confrontation', 'eliminated',

Getting clean documents from txt files for NON-HYPERPARTISAN ...: 100%|██████████| 75000/75000 [05:51<00:00, 213.24it/s]


Clean document list (NON-HYPERPARTISAN) loaded successfully.
Saving NON-HYPERPARTISAN document list into pickle file ...
Pickle file saved. 

DOCUMENT LIST STATISTICS (NON-HYPERPARTISAN):
Number of documents: 75000
Document sample after performing preprocessing:
['chief', 'juvenile', 'probation', 'officer', 'gwendolyn', 'tucker', ',', 'charge', 'city', 'department', 'incarcerates', 'attempts', 'rehabilitate', 'thousands', 'children', 'accused', 'committing', 'crimes', ',', 'facing', 'flood', 'criticism', 'youth', 'advocates', ',', 'staff', 'juvenile', 'probation', 'commissioners', 'considering', 'whether', 'fire', '.', 'juvenile', 'probation', 'commission', 'held', 'emergency', 'meeting', 'monday', 'lasted', 'long', 'night', 'reconvening', 'today', 'discuss', 'tucker', "'s", 'fate', '.', 'tucker', ',', 'promoted', 'assistant', 'chief', 'last', 'summer', 'then-', 'chief', 'jesse', 'williams', 'quit', ',', 'accused', 'ignoring', 'gas', 'leaks', 'fire', 'hazards', 'log', 'cabin', 'ranch',

### 2.4. Remove infrequent words (log-odd ratio is sensitive to them)

In [ ]:
hyperpartisan_document_list, non_hyperpartisan_document_list = (hyperpartisan_documents_processor.
    remove_infrequent_words(
        hyperpartisan_documents=hyperpartisan_document_list,
        non_hyperpartisan_documents=non_hyperpartisan_document_list
    )
)

### 2.5. Calculate log-odd ratios

#### 2.5.1. Calculate on `UNIGRAMS`

In [ ]:
console_output_formatter.print_section_header(section_header='Log-odd ratios calculation (on unigrams)')
log_odd_ratios_calculator = LogOddRatiosCalculator(
    hyperpartisan_documents=hyperpartisan_document_list,
    non_hyperpartisan_documents=non_hyperpartisan_document_list,
    token_type=TokenType.UNIGRAM
)

log_odd_ratios_calculator.calculate_log_odd_ratios()

#### 2.5.2. Calculate on `BIGRAMS`

In [ ]:
console_output_formatter.print_section_header(section_header='Log-odd ratios calculation (on bigrams)')
log_odd_ratios_calculator = LogOddRatiosCalculator(
    hyperpartisan_documents=hyperpartisan_document_list,
    non_hyperpartisan_documents=non_hyperpartisan_document_list,
    token_type=TokenType.BIGRAM
)

log_odd_ratios_calculator.calculate_log_odd_ratios()

### 2.6. Get most relevant words for each document group

#### 2.6.1. Most relevant words (on `UNIGRAMS`)

In [ ]:
console_output_formatter.print_section_header(section_header='Most relevant words extraction (on unigrams)')
log_odd_ratios_analyzer = LogOddRatiosAnalyzer(token_type=TokenType.UNIGRAM)

hyperpartisan_most_relevant_words_on_unigrams = (log_odd_ratios_analyzer.get_most_relevant_words(document_type=DocumentType.HYPERPARTISAN))
print(f'{hyperpartisan_most_relevant_words_on_unigrams} \n')
hyperpartisan_most_relevant_words_not_inf_on_unigrams = (log_odd_ratios_analyzer.
    get_most_relevant_words(
        document_type=DocumentType.HYPERPARTISAN,
        infinite_values=False
    )
)
print(f'{hyperpartisan_most_relevant_words_not_inf_on_unigrams} \n\n')

non_hyperpartisan_most_relevant_words_on_unigrams = (log_odd_ratios_analyzer.get_most_relevant_words(document_type=DocumentType.NON_HYPERPARTISAN))
print(f'{non_hyperpartisan_most_relevant_words_on_unigrams} \n')
non_hyperpartisan_most_relevant_words_not_inf_on_unigrams = (log_odd_ratios_analyzer.
    get_most_relevant_words(
        document_type=DocumentType.NON_HYPERPARTISAN,
        infinite_values=False
    )
)
print(non_hyperpartisan_most_relevant_words_not_inf_on_unigrams)

#### 2.6.2. Most relevant words (on `BIGRAMS`)

In [ ]:
console_output_formatter.print_section_header(section_header='Most relevant words extraction (on bigrams)')
log_odd_ratios_analyzer = LogOddRatiosAnalyzer(token_type=TokenType.BIGRAM)

hyperpartisan_most_relevant_words_on_bigrams = (log_odd_ratios_analyzer.get_most_relevant_words(document_type=DocumentType.HYPERPARTISAN))
print(f'{hyperpartisan_most_relevant_words_on_bigrams} \n')
hyperpartisan_most_relevant_words_not_inf_on_bigrams = (log_odd_ratios_analyzer.
    get_most_relevant_words(
        document_type=DocumentType.HYPERPARTISAN,
        infinite_values=False
    )
)
print(f'{hyperpartisan_most_relevant_words_not_inf_on_bigrams} \n\n')

non_hyperpartisan_most_relevant_words_on_bigrams = (log_odd_ratios_analyzer.get_most_relevant_words(document_type=DocumentType.NON_HYPERPARTISAN))
print(f'{non_hyperpartisan_most_relevant_words_on_bigrams} \n')
non_hyperpartisan_most_relevant_words_not_inf_on_bigrams = (log_odd_ratios_analyzer.
    get_most_relevant_words(
        document_type=DocumentType.NON_HYPERPARTISAN,
        infinite_values=False
    )
)
print(non_hyperpartisan_most_relevant_words_not_inf_on_bigrams)

\* NOTE: Due to excessive RAM memory usage some checkpoints have been stored via *pickle* files and the environment has been restarted 2-3 times. 

## 3. Results Analysis